In [ ]:
!pip install buzzard

In [ ]:
!pip install segmentation-models-pytorch

In [ ]:
import numpy as np
import buzzard as buzz
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import tqdm
import os

In [ ]:
import numpy as np
from tensorflow.keras.utils import Sequence
import cv2 as cv

In [ ]:
image_size = 256
data_folder = '../input/inria-aerial-image-labeling-dataset/AerialImageDataset'
train_folder_final='./train'

src_train_folder = os.path.join(data_folder, 'train', 'images')
src_train_folder_gt = os.path.join(data_folder, 'train', 'gt')
src_test_folder = os.path.join(data_folder, 'test', 'images')

src_train_images = os.listdir(src_train_folder)
src_test_images = os.listdir(src_test_folder)

train_folder_root = os.path.join(train_folder_final.format(image_size, image_size))
train_folder = os.path.join(train_folder_root, 'images')
train_folder_gt = os.path.join(train_folder_root, 'gt')


def create_gaussian(size=image_size, sigma=0.55):
    x, y = np.meshgrid(np.linspace(-1, 1, size), np.linspace(-1, 1, size))
    d = np.sqrt(x * x + y * y)
    gaussian = np.exp(-(d ** 2 / (2.0 * sigma ** 2)))
    return gaussian


class DataAugmentation(Sequence):

    def __init__(self, batch_size, validation, validation_set, process_input, border, debug=False):
        assert(0 <= validation_set <= 6)
        self.batch_size = batch_size
        self.validation = validation
        self.validation_set = validation_set
        self.process_input = process_input
        self.border = border
        self.debug = debug

        if self.debug:
            if not os.path.exists(debug_folder):
                os.makedirs(debug_folder)

        # Build image list
        self.images = []
        for fname in os.listdir(train_folder):
            name = fname.split('_')[0]
            i = len(name) - 1
            while name[i].isdigit():
                i -= 1
            i += 1
            n = int(name[i:])
            if validation_set > 0:
                if self.validation:
                    if (n - 1) // 6 == self.validation_set - 1:
                        self.images.append(fname)
                else:
                    if (n - 1) // 6 != self.validation_set - 1:
                        self.images.append(fname)
            elif not self.validation:
                self.images.append(fname)

        # Shuffle data
        if self.validation:
            self.images = np.random.RandomState(0).permutation(self.images)
            print("validation_elements = " + str(len(self.images)))
        else:
            self.images = np.random.RandomState(0).permutation(self.images)
            print("training_elements = " + str(len(self.images)))

        # Create border structuring element
        if self.border:
            self.structuring_element = cv.getStructuringElement(cv.MORPH_RECT, (3, 3))

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, idx):
        batch_start = idx * self.batch_size
        batch_end = min(len(self.images), (idx + 1) * self.batch_size)
        batch_images = self.images[batch_start:batch_end]

        batch_x = np.zeros((len(batch_images), image_size, image_size, 3), dtype=np.float32)
        if self.border:
            batch_y = np.zeros((len(batch_images), image_size, image_size, 2), dtype=np.float32)
        else:
            batch_y = np.zeros((len(batch_images), image_size, image_size, 1), dtype=np.float32)

        for i in range(len(batch_images)):
            fname = batch_images[i]
            fpath = os.path.join(train_folder, fname)
            fpath_gt = os.path.join(train_folder_gt, fname[:-4] + '.png')
            image = cv.imread(fpath)
            image_gt = cv.imread(fpath_gt, 0)
            image_gt = np.expand_dims(image_gt, -1)

            if not self.validation:
                t = self.get_random_transform()
                image = self.transform(image, t)
                image_gt = self.transform(image_gt, t)

            batch_x[i] = self.process_input(image)

            if self.border:
                border = cv.dilate(image_gt, self.structuring_element) - cv.erode(image_gt, self.structuring_element)
                border = np.reshape(border, (image_size, image_size, 1))
                batch_y[i] = np.concatenate((image_gt, border), axis=-1) / 255
            else:
                batch_y[i] = image_gt / 255

            if self.debug:
                cv.imwrite(os.path.join(debug_folder, fname), image)
                cv.imwrite(os.path.join(debug_folder, fname[:-4] + '.png'), image_gt)
                if self.border:
                    cv.imwrite(os.path.join(debug_folder, fname[:-4] + '_b.png'), border)

        return batch_x, batch_y

    @staticmethod
    def get_random_transform():
        tc = 6
        t = min(tc-1, int(np.floor(tc * np.random.rand())))
        return t

    @staticmethod
    def transform(img, t):
        if t == 1:
            return np.fliplr(img)
        if t == 2:
            return np.flipud(img)
        if t == 3:
            return np.rot90(img, 2)
        if t == 4:
            return np.rot90(img, -1)
        if t == 5:
            return np.rot90(img, 1)
        return img

    @staticmethod
    def inverse_transform(img, t):
        if t == 1:
            return np.fliplr(img)
        if t == 2:
            return np.flipud(img)
        if t == 3:
            return np.rot90(img, -2)
        if t == 4:
            return np.rot90(img, 1)
        if t == 5:
            return np.rot90(img, -1)
        return img


def test_data_augmentation():
    img = cv.imread(os.path.join(train_folder, 'austin1_9_0.jpg'))
    for i in range(100):
        t = DataAugmentation.get_random_transform()
        img_aug = DataAugmentation.transform(img, t)
        img_aug = np.clip(img_aug, 0, 255).astype(np.uint8)
        cv.imwrite(os.path.join(tmp_folder, str(i) + '.jpg'), img_aug)

In [ ]:
import numpy as np
import cv2 as cv
import shutil
import math
master_size = 1000
overlap = 0.3
threshold = 0.45

test = False

count = math.ceil((master_size - image_size * overlap) / (image_size * (1 - overlap)))
step = (master_size - image_size * overlap) / count
print('count =', count, ', step =', step)

if not os.path.exists(train_folder_root):
    os.makedirs(train_folder_root)

if not os.path.exists(train_folder):
    os.makedirs(train_folder)
else:
    shutil.rmtree(train_folder)

if not os.path.exists(train_folder_gt):
    os.makedirs(train_folder_gt)
else:
    shutil.rmtree(train_folder_gt)

for filename in src_train_images:
    print(filename)
    master_img = cv.imread(os.path.join(src_train_folder, filename))
    master_img_gt = cv.imread(os.path.join(src_train_folder_gt, filename))

    for i in range(count):
        if i < count - 1:
            y = round(i * step)
        else:
            y = master_size - image_size

        for j in range(count):
            if j < count - 1:
                x = round(j * step)
            else:
                x = master_size - image_size

            img = master_img[y:y+image_size, x:x+image_size]
            img_gt = master_img_gt[y:y+image_size, x:x+image_size]

            img_fname = '{}_{}_{}.{}'.format(filename[:-4], i, j, 'jpg')
            img_gt_fname = '{}_{}_{}.{}'.format(filename[:-4], i, j, 'png')
            cv.imwrite(os.path.join(train_folder, img_fname), img)
            cv.imwrite(os.path.join(train_folder_gt, img_gt_fname), img_gt)

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
image = cv2.imread("./train/images/austin1_1_0.jpg")
plt.imshow(image)
plt.show()

In [ ]:
!pip install torchsummary

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader
from torchvision import transforms as T
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable

from PIL import Image
import cv2
import albumentations as A

import time 
from tqdm.notebook import tqdm
from torchsummary import summary
import segmentation_models_pytorch as smp

device =torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
images= "./train/images/"
targets= "./train/gt/"

n_classes = 2

def _df_():
  name = []
  for dirname ,_ , filenames in os.walk(images):
    for filename in filenames:
      name.append(filename.split('.')[0])
    
    return pd.DataFrame({'id':name} , index = np.arange(0, len(name)))

df = _df_()
print(len(df))

In [ ]:
X_train , X_val = train_test_split(df['id'].values , test_size
                                   =0.25, random_state =19)
print(f'Train : {len(X_train)}')
print(f'Val : {len(X_val)}')

In [ ]:
img = Image.open(images + df['id'][0] + '.jpg')
targ = Image.open(targets + df['id'][0] + '.png')

print('Image Size', np.asarray(img).shape)
print('Mask Size' , np.asarray(targ).shape)

plt.figure(figsize=(10,10))
plt.imshow(img)
plt.imshow(targ , alpha =0.4)
plt.show()

In [ ]:
def dense_target(tar: np.ndarray):
    classes =np.unique(tar)
    dummy= np.zeros_like(tar)
    for idx , value in enumerate(classes):
        mask = np.where(tar == value)
        dummy[mask] = idx
    return dummy

class SegData(Dataset):

  def __init__(self , image_path , target_path , X , mean , std , transform =None , test=False, second=False):
    self.image_path = image_path
    self.target_path = target_path
    self.X = X
    self.transform =transform
    self.mean = mean
    self.std = std
    self.test =test
    self.second = second

  def __len__(self):
    return len(self.X)
  
  def __getitem__(self, idx):
    img = cv2.cvtColor(cv2.imread(self.image_path + self.X[idx] + '.jpg') , cv2.COLOR_BGR2RGB)
    target = cv2.imread(self.target_path + self.X[idx] + '.png' , cv2.IMREAD_GRAYSCALE)
    kernel_sharp = np.array(([-2, -2, -2], [-2, 17, -2], [-2, -2, -2]), dtype='int')
    img = cv2.filter2D(img, -1, kernel_sharp)
    target = cv2.filter2D(target, -1, kernel_sharp)
    img = cv2.resize(img, (256 , 256) , interpolation = cv2.INTER_NEAREST)
    target = cv2.resize(target , (256 , 256), interpolation = cv2.INTER_NEAREST)
    target = np.where( target > 0,255,0)
  
    if self.transform is not None:
      aug = self.transform(image = img , target = target )
      img = Image.fromarray(aug['image'])
      target = aug['target']
    
    if self.transform is None:
      img = Image.fromarray(img) 
    
    t = T.Compose([T.ToTensor() , T.Normalize(self.mean , self.std)])
    
    if self.second is False:
        if self.test is False:
            img = t(img)
        target = dense_target(target)
        target = torch.from_numpy(target).long()
        return img ,target
    
    if self.second is True:
        if self.test is False:
            img = t(img)
            model.to(device) ; image = img.to(device)
            with torch.no_grad():
                image = image.unsqueeze(0)
                output = model(image)
                masked = torch.argmax(output , dim =1)
                masked = masked.cpu().squeeze(0)
        return img, masked
    

In [ ]:
mean = [0.485 ,0.456 ,0.406]
std = [0.229 , 0.224 , 0.225]

train_set = SegData(images, targets, X_train , mean, std)
val_set = SegData(images , targets , X_val , mean , std)

batch_size = 4
train_loader= DataLoader(train_set , batch_size= batch_size , shuffle =True)
val_loader = DataLoader(val_set , batch_size = batch_size , shuffle =True)

In [ ]:
x , y =next(iter(train_loader))

print(f' x = shape : {x.shape} ; type :{x.dtype}')
print(f' x = min : {x.min()} ; max : {x.max()}')
print(f' y = shape: {y.shape}; class : {y.unique()}; type: {y.dtype}')

In [ ]:
model = smp.Unet('resnet34',encoder_weights='imagenet', classes = 2, activation=None,
                 encoder_depth= 5, decoder_channels=[256,128, 64, 32,16])
model=model.to(device)

In [ ]:
summary(model, input_size=(3, 256 , 256))

In [ ]:
def pixel_wise_accuracy(output , mask):
  with torch.no_grad():
    output = torch.argmax(F.softmax(output , dim =1) , dim=1)
    correct = torch.eq(output , mask).int()
    accuracy = float(correct.sum())/ float(correct.numel())#total number
  return accuracy

In [ ]:
def IoU(pred , true_pred , smooth =1e-10 , n_classes=2):
  with torch.no_grad():
    pred = torch.argmax(F.softmax(pred , dim =1) , dim=1)
    pred = pred.contiguous().view(-1)
    true_pred = true_pred.contiguous().view(-1)

    iou_class = []
    for value in range(0, n_classes):
      true_class = pred == value
      true_label = true_pred == value

      if true_label.long().sum().item()==0:
        iou_class.append(np.nan)
        
      else:
    
        inter = torch.logical_and(true_class, true_label).sum().float().item()
        union = torch.logical_or(true_class , true_label).sum().float().item()

        iou = (inter + smooth)/(union + smooth)
        iou_class.append(iou)

    return np.nanmean(iou_class)

In [ ]:
def DiceBceLoss(true, logits, eps=1e-7):
    num_classes = logits.shape[1]
    if num_classes == 1:
        true_1_hot = torch.eye(num_classes + 1)[true.squeeze(1)]
        true_1_hot = true_1_hot.permute(0, 3, 1, 2).float()
        true_1_hot_f = true_1_hot[:, 0:1, :, :]
        true_1_hot_s = true_1_hot[:, 1:2, :, :]
        true_1_hot = torch.cat([true_1_hot_s, true_1_hot_f], dim=1)
        pos_prob = torch.sigmoid(logits)
        neg_prob = 1 - pos_prob
        probas = torch.cat([pos_prob, neg_prob], dim=1)
    else:
        true_1_hot = torch.eye(num_classes)[true.squeeze(1)]
        true_1_hot = true_1_hot.permute(0, 3, 1, 2).float()
        probas = F.softmax(logits, dim=1)
    true_1_hot = true_1_hot.type(logits.type())
    dims = (0,) + tuple(range(2, true.ndimension()))
    intersection = torch.sum(probas * true_1_hot, dims)
    cardinality = torch.sum(probas + true_1_hot, dims)
    dice_loss = 1- ((2.*intersection + eps)/(cardinality + eps)).mean()
    bce = F.cross_entropy(logits, true , reduction ="mean")
    dice_bce = bce + dice_loss
    return dice_bce

In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit(epochs, model, train_loader, val_loader, optimizer, scheduler, patch=False):
    train_losses = []
    test_losses = []
    val_iou = []; val_acc = []
    train_iou = []; train_acc = []
    lrs = []
    min_loss = np.inf
    decrease = 1 ; not_improve=0

    model.to(device)
    fit_time = time.time()
    for e in range(epochs):
        since = time.time()
        running_loss = 0
        iou_score = 0
        accuracy = 0
        #training loop
        model.train()
        for i, data in enumerate(tqdm(train_loader)):
            #training phase
            image_tiles, mask_tiles = data
            if patch:
                bs, n_tiles, c, h, w = image_tiles.size()

                image_tiles = image_tiles.view(-1,c, h, w)
                mask_tiles = mask_tiles.view(-1, h, w)
            
            image = image_tiles.to(device); mask = mask_tiles.to(device);
            #forward
            output = model(image)
            loss = DiceBceLoss(mask, output)
            #evaluation metrics
            iou_score += IoU(output, mask)
            accuracy += pixel_wise_accuracy(output, mask)
            #backward
            loss.backward()
            optimizer.step() #update weight          
            optimizer.zero_grad() #reset gradient
            
            #step the learning rate
            lrs.append(get_lr(optimizer))
            scheduler.step() 
            
            running_loss += loss.item()
            
        else:
            model.eval()
            test_loss = 0
            test_accuracy = 0
            val_iou_score = 0
            #validation loop
            with torch.no_grad():
                for i, data in enumerate(tqdm(val_loader)):
                    #reshape to 9 patches from single image, delete batch size
                    image_tiles, mask_tiles = data

                    if patch:
                        bs, n_tiles, c, h, w = image_tiles.size()

                        image_tiles = image_tiles.view(-1,c, h, w)
                        mask_tiles = mask_tiles.view(-1, h, w)
                    
                    image = image_tiles.to(device); mask = mask_tiles.to(device);
                    output = model(image)
                    #evaluation metrics
                    val_iou_score +=  IoU(output, mask)
                    test_accuracy += pixel_wise_accuracy(output, mask)
                    #loss
                    loss = DiceBceLoss(mask, output)                                  
                    test_loss += loss.item()
            
            #calculatio mean for each batch
            train_losses.append(running_loss/len(train_loader))
            test_losses.append(test_loss/len(val_loader))


            if min_loss > (test_loss/len(val_loader)):
                print('Loss Decreasing.. {:.3f} >> {:.3f} '.format(min_loss, (test_loss/len(val_loader))))
                min_loss = (test_loss/len(val_loader))
                decrease += 1
                print('saving model...')
                torch.save(model, 'model-{:.3f}.pt'.format(val_iou_score/len(val_loader)))
                    

            # if (test_loss/len(val_loader)) > min_loss:
            #     not_improve += 1
            #     min_loss = (test_loss/len(val_loader))
            #     print(f'Loss did not  Decrease for {not_improve} time')
            #     if not_improve == 7:
            #         print('Loss did not decrease for the 7th time , Stop Training')
            #         break
            
            #iou
            val_iou.append(val_iou_score/len(val_loader))
            train_iou.append(iou_score/len(train_loader))
            train_acc.append(accuracy/len(train_loader))
            val_acc.append(test_accuracy/ len(val_loader))
            print("Epoch:{}/{}..".format(e+1, epochs),
                  "Train Loss: {:.3f}..".format(running_loss/len(train_loader)),
                  "Val Loss: {:.3f}..".format(test_loss/len(val_loader)),
                  "Train IoU:{:.3f}..".format(iou_score/len(train_loader)),
                  "Val IoU: {:.3f}..".format(val_iou_score/len(val_loader)),
                  "Train Acc:{:.3f}..".format(accuracy/len(train_loader)),
                  "Val Acc:{:.3f}..".format(test_accuracy/len(val_loader)),
                  "Time: {:.2f}m".format((time.time()-since)/60))
        
    history = {'train_loss' : train_losses, 'val_loss': test_losses,
               'train_miou' :train_iou, 'val_miou':val_iou,
               'train_acc' :train_acc, 'val_acc':val_acc,
               'lrs': lrs}
    print('Total time: {:.2f} m' .format((time.time()- fit_time)/60))
    return history

In [ ]:
max_lr = 1e-3
epoch = 20
weight_decay = 1e-6

optimizer = torch.optim.Adam(model.parameters(), lr=max_lr, weight_decay=weight_decay)
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epoch,
                                            steps_per_epoch=len(train_loader))

history = fit(epoch, model, train_loader, val_loader, optimizer, sched)

In [ ]:
test_set = SegData(images, targets, X_val ,mean , std, transform = None , test = True)

In [ ]:
def predict_image_mask(model, image , mask , mean=[0.485, 0.456, 0.406],
                       std = [0.229 , 0.224 ,0.225]):
  model.eval()
  t= T.Compose([T.ToTensor() ,T.Normalize(mean, std)])
  image = t(image)
  model.to(device) ; image = image.to(device)
  mask = mask.to(device)
  with torch.no_grad():

    image = image.unsqueeze(0)
    mask = mask.unsqueeze(0)

    output = model(image)
    score = IoU(output, mask)
    masked = torch.argmax(output , dim =1)
    masked = masked.cpu().squeeze(0)
  return masked , score

In [ ]:
import random
for i in range(10,20):
  image , mask = test_set[i]
  pred_mask , score = predict_image_mask(model , image , mask)
  fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(20,10))
  ax1.imshow(image)
  ax1.set_title('Picture');

  ax2.imshow(mask)
  ax2.set_title('Ground truth')
  ax2.set_axis_off()

  ax3.imshow(pred_mask)
  ax3.set_title('UNet-Resnet50 | DiceBCE {:.3f}'.format(score))
  ax3.set_axis_off()

In [ ]:
print("Start send round!")

In [ ]:
mean = [0.485 ,0.456 ,0.406]
std = [0.229 , 0.224 , 0.225]

train_set2 = SegData(images, targets, X_train , mean, std, second = True)
val_set2 = SegData(images , targets , X_val , mean , std, second = True)

batch_size = 4
train_loader2 = DataLoader(train_set2 , batch_size= batch_size , shuffle =True)
val_loader2 = DataLoader(val_set2 , batch_size = batch_size , shuffle =True)

In [ ]:
x , y =next(iter(train_loader2))

print(f' x = shape : {x.shape} ; type :{x.dtype}')
print(f' x = min : {x.min()} ; max : {x.max()}')
print(f' y = shape: {y.shape}; class : {y.unique()}; type: {y.dtype}')

In [ ]:
model2 = smp.Unet('resnet34',encoder_weights='imagenet', classes = 2, activation=None,
                 encoder_depth= 5, decoder_channels=[256,128, 64, 32,16])
model2=model2.to(device)

In [ ]:
summary(model2, input_size=(3, 256 , 256))

In [ ]:
max_lr = 1e-3
epoch = 20
weight_decay = 1e-6

optimizer = torch.optim.Adam(model2.parameters(), lr=max_lr, weight_decay=weight_decay)
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epoch,
                                            steps_per_epoch=len(train_loader2))

history = fit(epoch, model2, train_loader2, val_loader2, optimizer, sched)

In [ ]:
test_set2 = SegData(images, targets, X_val ,mean , std, transform = None , test = True)

In [ ]:
def predict_image_mask2(model, image , mask , mean=[0.485, 0.456, 0.406],
                       std = [0.229 , 0.224 ,0.225]):
  model.eval()
  t= T.Compose([T.ToTensor() ,T.Normalize(mean, std)])
  image = t(image)
  model.to(device) ; image = image.to(device)
  mask = mask.to(device)
  with torch.no_grad():

    image = image.unsqueeze(0)
    mask = mask.unsqueeze(0)

    output = model(image)
    score = IoU(output, mask)
    masked = torch.argmax(output , dim =1)
    masked = masked.cpu().squeeze(0)
  return masked , score

In [ ]:
import random
for i in range(10,20):
  image , mask = test_set2[i]
  pred_mask , score = predict_image_mask(model2 , image , mask)
  fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(20,10))
  ax1.imshow(image)
  ax1.set_title('Picture');

  ax2.imshow(mask)
  ax2.set_title('Ground truth')
  ax2.set_axis_off()

  ax3.imshow(pred_mask)
  ax3.set_title('UNet-Resnet50 | DiceBCE {:.3f}'.format(score))
  ax3.set_axis_off()